In [204]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy import linalg
import matplotlib.pyplot as plt
import random
from IPython.display import display, clear_output
from ipywidgets import widgets, Layout, interact, interactive, fixed, interact_manual
from jupyterthemes import jtplot
import time
import scipy as scipy
import csv

%run ~/Documents/STEIN_links/for_publication/functions_for_wrapper.ipynb

In [205]:
# Input the initial variables:
style = {'description_width' : 'initial'}
erosion_type = widgets.Dropdown(options = [('Stochastic Rockfalls', 'stochastic'),
                                           ('Steady-state', 'constant'),
                                           ('No Erosion', 'none')],
                                value='stochastic',
                                description='Erosion type:',
                                style = style)

print('Select an erosion type for the simulation.')
display(erosion_type)

total_time = widgets.BoundedIntText(value = 75,
                                   min = 10,
                                   max = 150000,
                                   description = "Total time:")

print('Input total time length (years) for individual simulations. \nMax = 150000 \nMin = 10')
display(total_time)

SLHL_C14 = widgets.Dropdown(options=[('9.5 \t (Fenton et al. (2019)', 9.5),
                                      ('12.76 \t (Borchers et al. (2016)', 12.76)],
                           value = 12.76,
                           description='$^{14}$C Production Rate:',
                           style = style)
print('Select a C-14 sea level high latitude production rate. \nBorchers et al. (2016) (12.76) is recommended.')
display(SLHL_C14)

SLHL_Be10 = widgets.Dropdown(options=[('3.83 \t (Borchers et al. (2016)', 3.83),
                                      ('3.89 \t (Borchers et al. (2016)', 3.89),
                                      ('3.93 \t (Borchers et al. (2016)', 3.93),
                                      ('4.02 \t (Borchers et al. (2016))', 4.02)],
                           value=3.89,
                           description='$^{10}$Be Production Rate:',
                           style = style)
print('Select a Be-10 sea level high latitude production rate.')
display(SLHL_Be10)

SLHL_He3 = widgets.Dropdown(options=[('102 \t ( Vermeesh et al. (2009)', 102.0),
                                      ('107 \t (Vermeesh et al. (2009)', 107.0),
                                      ('110 \t (Vermeesh et al. (2009)', 110.0),
                                     ('112.26 \t (Borchers et al. (2016)', 112.26),
                                      ('114.55 \t (Borchers et al. (2016)', 114.55),
                                     ('117.47 \t (Borchers et al. (2016)', 117.47),
                                     ('128 \t (Vermeesh et al. (2009))', 128.0)],
                           value=114.55,
                           description='$^{3}$He Production Rate:',
                           style = style)
print('Select a He-3 sea level high latitude production rate.')
display(SLHL_He3)

scaling_factor = widgets.BoundedFloatText(value = 1.0,
                                min = 0.0000001,
                                max = 900.0,
                                step = 0.1, 
                                description = "CN scaling factor: \t",
                                style = style)
print('Input cosmogenic nuclide production rate scaling factor. \nFor SLHL, enter ' + str('1') + '.')
display(scaling_factor)

gr_sz = widgets.BoundedFloatText(value = 0.1,
                                min = 0.08,
                                max = 2.0,
                                step = 0.01, 
                                description = "Quartz grain size",
                                style = style)
print('Input the mean grain size of the quartz used for He-3 measurements.')
display(gr_sz)

MAT = widgets.BoundedFloatText(value = 0.0,
                                min = -50.0,
                                max = 15.0,
                                step = 0.5, 
                                description = "Mean annual temperature (0 $\circ$C)",
                                style = style)
print('Input a mean annual temperature for the simulations.')
display(MAT)

T_AMP=widgets.BoundedFloatText(value=5.0,
                                min=0.5,
                                max=35.0,
                                step=0.5, 
                                description = "Temperature seasonality",
                                style = style)
print('Input an amplitude for the seasonal temperature function. \nAmplitude of 5.0 for MAT = 0.0 indicates mean summer temperature of +5.0 C and mean winter temperature of -5.0 C.')
display(T_AMP)

save_output = widgets.Dropdown(options = [('Yes', True), ('No',False)],
                              value = True,
                              description = 'Save Output')
print('Would you like to save the output files?')
display(save_output)

save_only_surf = True

Select an erosion type for the simulation.


Dropdown(description='Erosion type:', options=(('Stochastic Rockfalls', 'stochastic'), ('Steady-state', 'const…

Input total time length (years) for individual simulations. 
Max = 150000 
Min = 10


BoundedIntText(value=75, description='Total time:', max=150000, min=10)

Select a C-14 sea level high latitude production rate. 
Borchers et al. (2016) (12.76) is recommended.


...

BoundedFloatText(value=0.0, description='Mean annual temperature (0 $\\circ$C)', max=15.0, min=-50.0, step=0.5…

Input an amplitude for the seasonal temperature function. 
Amplitude of 5.0 for MAT = 0.0 indicates mean summer temperature of +5.0 C and mean winter temperature of -5.0 C.


BoundedFloatText(value=5.0, description='Temperature seasonality', max=35.0, min=0.5, step=0.5, style=Descript…

Would you like to save the output files?


Dropdown(description='Save Output', options=(('Yes', True), ('No', False)), value=True)

In [206]:
# Input the case-dependent variables:
if erosion_type.value == 'stochastic':
    scenarios = widgets.BoundedIntText(value = 1,
                                   min = 2,
                                   max = 30,
                                   description = 'Scenarios:',
                                   style = style)
    print('Input number of stochastic erosion scenarios. \nMax = 30 \nMin = 1')
    display(scenarios)
    
    stoch_base_ER = widgets.BoundedFloatText(value = 0.1,
                                   min = 0.00001,
                                   max = 349.0,
                                   description = 'Base Erosion Rate (cm yr$^{-1}$:',
                                   style = style)
    print('Input the time-averaged erosion rate (total lost / total time).')
    display(stoch_base_ER)
    
    sample_no = widgets.BoundedIntText(value = 30,
                                      min = 2,
                                      max = (total_time.value - 1),
                                      description = 'No. of Sub-samples:',
                                      style = style)
    print('Input the number of sub-samples you would like to base the CN measurements on for each stochastic sample. These concentrations are selected from random times in the model run, as in a "time for space" substition.') 
    display(sample_no)
    
    pareto_c = widgets.BoundedIntText(value = 1.0,
                                      min = 0.1,
                                      max = 5,
                                      description = 'Pareto Distribution Shape Value',
                                      style = style)
    print('Input the Pareto distribution shape value (c).') 
    display(pareto_c)
    

if erosion_type.value == 'constant':
    erosion_rate = widgets.FloatText(value = 0.1,
                                     min = 0.001,
                                     max = 350,
                                     description = 'Erosion Rate (cm yr$^{-1}$)',
                                    style = style)
    print('Input the steady-state erosion rate.')
    display(erosion_rate)

    shift_ER = widgets.Dropdown(options = [('Yes', True), ('No', False)],
                               value = False,
                               description = 'Change erosion rate mid-simulation?',
                               style = style)

    print('Would you like to change the erosion rate during the simulation? This will be a step-function change at a specific time.')  
    display(shift_ER)

    time_ER_shift = widgets.BoundedIntText(value = int(0.5 * total_time.value),
                                          min = 0,
                                          max = (total_time.value - 1),
                                          description = 'Time for rate change:',
                                          style = style)
    new_ER = widgets.BoundedFloatText(value = 0.1,
                                      min = 0.0000001,
                                      max = 349,
                                      description = 'New erosion rate (cm yr$^{-1}$)',
                                     style = style)
    print('If you will change the erosion rate, please input the year of the time change and the new erosion rate.')
    display(time_ER_shift)
    display(new_ER)

Input number of stochastic erosion scenarios. 
Max = 30 
Min = 1


BoundedIntText(value=2, description='Scenarios:', max=30, min=2, style=DescriptionStyle(description_width='ini…

Input the time-averaged erosion rate (total lost / total time).


BoundedFloatText(value=0.1, description='Base Erosion Rate (cm yr$^{-1}$:', max=349.0, min=1e-05, style=Descri…

Input the number of sub-samples you would like to base the CN measurements on for each stochastic sample. These concentrations are selected from random times in the model run, as in a "time for space" substition.


BoundedIntText(value=30, description='No. of Sub-samples:', max=74, min=2, style=DescriptionStyle(description_…

Input the Pareto distribution shape value (c).


BoundedIntText(value=1, description='Pareto Distribution Shape Value', max=5, style=DescriptionStyle(descripti…

In [207]:
if get_values(erosion_type) == 'stochastic':
    [erosion_type, 
     total_time, 
     SLHL_C14, 
     SLHL_Be10,
     SLHL_He3, 
     scaling_factor, 
     gr_sz, MAT, T_AMP, 
     save_output,
     scenarios,
     stoch_base_ER, sample_no,
     pareto_c,
     max_depth, 
     rock_rho, 
     CN_lambda, 
     mu, dt, rho, 
     nx, r, dx, x, 
     x_up, x_low, 
     x_vol, shell_mass, 
     alpha, P0_Be10, P0_He3] = [set_values()[i] for i in range(len(set_values()))]
    
    %run ~\Documents\STEIN_links\for_publication\rockfall_generation_stochastic_erosion.ipynb    
    %run ~\Documents\STEIN_links\for_publication\C14_Be10_stochastic_erosion.ipynb
    %run ~\Documents\STEIN_links\for_publication\He3_stochastic_erosion.ipynb
    %run ~\Documents\STEIN_links\for_publication\stochastic_measured_constants_CRN_loops.ipynb

    rockfall_matrix_gen(stoch_base_ER, pareto_c)
    rockfall_output = erosion_export(stoch_base_ER)
        
    rockfalls_imported = rockfalls_import()
    CRN_output = CRN_loop_fxn(total_time)
    
    He3_output = He3_loop_fxn(total_time)

    comparison_constants, samples, measured, measured_ER = compare_fxn()
    og_ER_mat = original_ER_fxn(rockfalls_imported[1])
    master_array, master_df, master_means = post_process(CRN_output, He3_output)
    
    export_inputs = export_var_function()
    
    print('Completed Erosion Rate: ' + str(stoch_base_ER))

elif get_values(erosion_type) == 'constant':
     
    [erosion_type, 
     total_time, 
     SLHL_C14, 
     SLHL_Be10,
     SLHL_He3, 
     scaling_factor, 
     gr_sz, MAT, T_AMP, 
     save_output, initial_ER, 
     shift_ER, time_ER_shift,
     ER_shift_factor,
     max_depth, 
     rock_rho, 
     CN_lambda, 
     mu, dt, rho, 
     nx, r, dx, x, 
     x_up, x_low, 
     x_vol, shell_mass, 
     alpha, P0_Be10, P0_He3] = [set_values()[i] for i in range(len(set_values()))]    
    
    %run D:\STEIN_paper\model_outputs\scripts\C14_Be10_constant_erosion.ipynb
    CRN_output = CRN_loop_fxn(total_time, time_ER_shift)
    
    %run D:\STEIN_paper\model_outputs\scripts\He3_constant_erosion.ipynb
    %run D:\STEIN_paper\model_outputs\scripts\He3_constant_erosion_kinparams_test.ipynb

    He3_output = He3_loop_fxn(total_time, time_ER_shift)
    
    He3_percents = np.divide((He3_output[1]),CRN_output[4]) * P0_Be10 / P0_He3

elif get_values(erosion_type) == 'none':

    [erosion_type,
     total_time,
     SLHL_C14, 
     SLHL_Be10,
     SLHL_He3,
     scaling_factor, 
     gr_sz, 
     MAT, 
     T_AMP, 
     save_output] = [set_values()[i] for i in range(len(set_values()))]
    
    %run D:\STEIN_paper\model_outputs\scripts\C14_Be10_no_erosion.ipynb
    CRN_output = CRN_loop_fxn(total_time, time_ER_shift)
    
    %run D:\STEIN_paper\model_outputs\scripts\He3_no_erosion.ipynb
    He3_output = He3_loop_fxn(total_time, time_ER_shift)
    
    He3_percents = np.divide((He3_output[1]),CRN_output[4]) * P0_Be10 / P0_He3

Main C14 & Be10 loop scenario 0complete.
Main C14 & Be10 loop scenario 1complete.
Main He3 loop scenario 74complete.
Main He3 loop scenario 74complete.


C:\Users\dennis\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning:

invalid value encountered in true_divide



Comparison Be10 / C14 loop complete.
Comparison He3 loop scenario complete.


C:\Users\dennis\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning:

invalid value encountered in true_divide



Comparison Be10 / C14 loop complete.
Comparison He3 loop scenario complete.


C:\Users\dennis\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning:

invalid value encountered in true_divide



Comparison Be10 / C14 loop complete.
Comparison He3 loop scenario complete.


C:\Users\dennis\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning:

invalid value encountered in true_divide



Comparison Be10 / C14 loop complete.
Comparison He3 loop scenario complete.
Completed Erosion Rate: 0.1


C:\Users\dennis\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

C:\Users\dennis\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

